In [1]:
! pip install numpy==1.22.4 pandas==1.5.3
! pip install pgvector==0.1.8
! pip install langchain==0.0.196 transformers==4.30.1
! pip install google-cloud-aiplatform==1.26.0
! pip install psycopg2-binary
! pip install protobuf==3.20.3
! pip install shapely==2.0.6

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 83.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 94.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires grpcio!=1.48.0,<2,>=1.33.1, but you have grpcio 1.48.0 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.5 which is incompatible.
contourpy 1.3.0 requires numpy>=1.23, but you have numpy 1.22.4 which is incompatible.
dataproc-jupyter-plugin 0.1.80 requires aiohttp~=3.9.5, but you h

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import os
import pandas as pd

In [2]:
import psycopg2

# Replace with your AlloyDB cluster credentials
cluster_ip_address = "10.111.0.2"
database_user = "postgres"
database_password = "postgres"

# Set environment variables for psql connection
os.environ["PGHOST"] = cluster_ip_address
os.environ["PGUSER"] = database_user
os.environ["PGPASSWORD"] = database_password

# Establish a connection to the database
try:
    conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password
    )
    print("Connected to the database successfully!")
except Exception as e:
    print("Connection error:", e)
    exit(1)

# Read the dataset from the URL
DATASET_URL = "https://github.com/GoogleCloudPlatform/python-docs-samples/raw/main/cloud-sql/postgres/pgvector/data/retail_toy_dataset.csv"
df = pd.read_csv(DATASET_URL)

# Select desired columns and drop missing values
df = df.loc[:, ["product_id", "product_name", "description", "list_price"]]
df = df.dropna()

# Save the DataFrame to the AlloyDB cluster
df.to_sql('products', con=f'postgresql://{cluster_ip_address}', if_exists='replace', index=False)

# Retrieve data from the 'products' table
cur = conn.cursor()
cur.execute("SELECT * FROM products")
results = cur.fetchall()

# Close the connection
conn.close()
print(results[5])

Connected to the database successfully!
('74a695e3675efc2aad11ed73c46db29b', 'Slip N Slide Triple Racer with Slide Boogies', 'Triple Racer Slip and Slide with Boogie Boards. The unit is 16 foot long. The unit has 3 sliding lanes.', 37.21)


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=500,
    chunk_overlap=0,
    length_function=len,
)
max_documents = 60
chunked = []
for index, row in df.iterrows():
    product_id = row["product_id"]
    desc = row["description"]
    splits = text_splitter.create_documents([desc])
    if len(chunked) < max_documents:
        for s in splits:
            r = {"product_id": product_id, "content": s.page_content}
            chunked.append(r)
    else:
        break
print(len(chunked))

63


In [4]:
# Generate the vector embeddings for each chunk of text.
# This code snippet may run for a few minutes.

from langchain.embeddings import VertexAIEmbeddings
from google.cloud import aiplatform
import time

aiplatform.init(project=f"qwiklabs-gcp-02-be06f8b1cebc", location=f"us-central1")
embeddings_service = VertexAIEmbeddings()

# Helper function to retry failed API requests with exponential backoff.
def retry_with_backoff(func, *args, retry_delay=5, backoff_factor=2, **kwargs):
    max_attempts = 10
    retries = 0
    for i in range(max_attempts):
        try:
            return func(*args, **kwargs)
        except Exception as e:
            print(f"error: {e}")
            retries += 1
            wait = retry_delay * (backoff_factor**retries)
            print(f"Retry after waiting for {wait} seconds...")
            time.sleep(wait)

batch_size = 5
for i in range(0, len(chunked), batch_size):
    request = [x["content"] for x in chunked[i : i + batch_size]]
    response = retry_with_backoff(embeddings_service.embed_documents, request)
    # Store the retrieved vector embeddings for each chunk back.
    for x, e in zip(chunked[i : i + batch_size], response):
        x["embedding"] = e

# Store the generated embeddings in a pandas dataframe.
product_embeddings = pd.DataFrame(chunked)
product_embeddings.head()

,product_id,content,embedding
0,7e8697b5b7cdb5a40daf54caf1435cd5,"Rock, paper, scissors is a great way to resolv...","[0.040036752820014954, -0.025681596249341965, ..."
1,7e8697b5b7cdb5a40daf54caf1435cd5,"games, creating your own game, and friends and...","[0.01725427247583866, -0.002125497441738844, -..."
2,7de8b315b3cb91f3680eb5b88a20dcee,Turn any small bicycle into an instrument for ...,"[-0.004991121590137482, -0.06028855964541435, ..."
3,7de8b315b3cb91f3680eb5b88a20dcee,teel brackets stand up to heavy use. Customiza...,"[0.00582809979096055, -0.056259434670209885, 0..."
4,7de8b315b3cb91f3680eb5b88a20dcee,es.com. Follow ride Schwinn on: Twitter. Faceb...,"[0.0065319109708070755, -0.05318767577409744, ..."


In [5]:
import psycopg2
import numpy as np
from pgvector.psycopg2 import register_vector

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.111.0.2"
database_user = "postgres"
database_password = "postgres"

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Create cursor for executing SQL commands
cur = conn.cursor()

# Ensure vector extension is installed
cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

# Drop existing table (if it exists)
cur.execute("DROP TABLE IF EXISTS product_embeddings")

# Create the `product_embeddings` table
cur.execute("""
    CREATE TABLE product_embeddings(
        product_id VARCHAR(1024) NOT NULL PRIMARY KEY,
        content TEXT,
        embedding vector(768)
    )
""")

# Register the vector type
register_vector(conn)

# Store vector embeddings in the table
for index, row in product_embeddings.iterrows():
    cur.execute("SELECT EXISTS(SELECT 1 FROM product_embeddings WHERE product_id = %s)", (row["product_id"],))
    if not cur.fetchone()[0]:  # Product ID doesn't exist, insert
        cur.execute("INSERT INTO product_embeddings (product_id, content, embedding) VALUES (%s, %s, %s)", (row["product_id"], row["content"], row["embedding"]))
    else:  # Product ID exists, update
        cur.execute("UPDATE product_embeddings SET content = %s, embedding = %s WHERE product_id = %s", (row["content"], row["embedding"], row["product_id"]))


# Commit changes and close connection
conn.commit()
conn.close()
print("Created the 'product_embeddings' table successfully")

Created the 'product_embeddings' table successfully


In [6]:
import psycopg2
from pgvector.psycopg2 import register_vector

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.111.0.2"
database_user = "postgres"
database_password = "postgres"

m = 24
ef_construction = 100
operator = "vector_cosine_ops"

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Register the vector type
register_vector(conn)

# Create the HNSW index on the `product_embeddings` table
cur = conn.cursor()
cur.execute(
    f"""CREATE INDEX ON product_embeddings
        USING hnsw(embedding {operator})
        WITH (m = {m}, ef_construction = {ef_construction})
    """
)
conn.commit()

# Close the connection
conn.close()
print("Created an HNSW Index successfully")

Created an HNSW Index successfully


In [7]:
import psycopg2
from pgvector.psycopg2 import register_vector

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.111.0.2"
database_user = "postgres"
database_password = "postgres"

lists = 100
operator = "vector_cosine_ops"

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Register the vector type
register_vector(conn)

# Create the IVFFLAT index on the `product_embeddings` table
cur = conn.cursor()
cur.execute(
    f"""CREATE INDEX ON product_embeddings
        USING ivfflat(embedding {operator})
        WITH (lists = {lists})
    """
)
conn.commit()

# Close the connection
conn.close()
print("Created an IVFFLAT Index successfully")

Created an IVFFLAT Index successfully


In [9]:
import psycopg2
from pgvector.psycopg2 import register_vector
import pandas as pd

# AlloyDB cluster connection details (replace with your actual values)
cluster_ip_address = "10.111.0.2"
database_user = "postgres"
database_password = "postgres"

toy = "playing card games"
min_price = 25
max_price = 100

# Connect to AlloyDB cluster
conn = psycopg2.connect(
    host=cluster_ip_address,
    user=database_user,
    password=database_password
)

# Register the vector type
register_vector(conn)

# Get the query embedding
qe = embeddings_service.embed_query([toy])

# Perform the similarity search and filtering
cur = conn.cursor()
similarity_threshold = 0.1
num_matches = 50
# Pass 'qe' twice to match the number of placeholders in the query
cur.execute(
    """
    WITH vector_matches AS (
        SELECT product_id, 1 - (embedding <=> %s::vector) AS similarity
        FROM product_embeddings
        WHERE 1 - (embedding <=> %s::vector) > %s
        ORDER BY similarity DESC
        LIMIT %s
    )
    SELECT product_name, list_price, description
    FROM products
    WHERE product_id IN (SELECT product_id FROM vector_matches)
    AND list_price >= %s AND list_price <= %s
    """,
    (qe, qe, similarity_threshold, num_matches, min_price, max_price)
)
results = cur.fetchall()

# Process the results
matches = []
for r in results:
    try:
        list_price = round(float(r[2]), 2)  # Attempt conversion and rounding
    except ValueError:
        list_price = r[2] 
    matches.append({
        "product_name": r[0],
        "list_price": r[1],
        "description": r[2]
    })

# Display the results
matches_df = pd.DataFrame(matches)
print(matches_df.head(5))

# Close the connection
conn.close()

                                        product_name  list_price  \
0                    12"-20" Schwinn Training Wheels       28.17   
1       Slip N Slide Triple Racer with Slide Boogies       37.21   
2  Polaris 39-310 5-Liter Zippered Super Bag for ...       39.47   
3  Sandbox Castle 2-in-1 Sand and Water Table wit...       60.49   
4  Jensen S100T Commercial Tot Full Bucket Rubber...       90.18   

                                         description  
0  Turn any small bicycle into an instrument for ...  
1  Triple Racer Slip and Slide with Boogie Boards...  
2  Keep your pool water sparkling clean all seaso...  
3  Package Includes Sandbox Castle 2-in-1 Sand an...  
4  This is a fully enclosed one piece infant seat...  


In [10]:
# Please fill in these values.
user_query = "Do you have a toy set that teaches numbers and letters to kids?"  # @param {type:"string"}
min_price = 20  # @param {type:"integer"}
max_price = 100  # @param {type:"integer"}

In [11]:
qe = embeddings_service.embed_query([user_query])

In [12]:
import psycopg2
from psycopg2 import sql
from pgvector.psycopg2 import register_vector

def main(user_query,min_price,max_price):
    try:
        # AlloyDB cluster connection details (replace with your actual values)
        cluster_ip_address = "10.111.0.2"
        database_user = "postgres"
        database_password = "postgres"
        
        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
        host=cluster_ip_address,
        user=database_user,
        password=database_password)

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings_service.embed_query([user_query])
        
        # Perform the similarity search and filtering
        cur = conn.cursor()
        similarity_threshold = 0.1
        num_matches = 50
        
        # Pass 'qe' twice to match the number of placeholders in the query
        cur.execute(
                    """
                        WITH vector_matches AS (
                        SELECT product_id, 1 - (embedding <=> %s::vector) AS similarity
                        FROM product_embeddings
                        WHERE 1 - (embedding <=> %s::vector) > %s
                        ORDER BY similarity DESC
                        LIMIT %s
                )
                SELECT product_name, list_price, description
                FROM products
                WHERE product_id IN (SELECT product_id FROM vector_matches)
                AND list_price >= %s AND list_price <= %s
                """,
                (qe, qe, similarity_threshold, num_matches, min_price, max_price))
        results = cur.fetchall()
        # Process the results
        matches = []
        for r in results:
            try:
                list_price = round(float(r[2]), 2)  # Attempt conversion and rounding
            except ValueError:
                    list_price = r[2] 
                    matches.append({
                        "product_name": r[0],
                        "list_price": r[1],
                        "description": r[2]
                    })
                    # Display the results
        matches_df = pd.DataFrame(matches)
        print(matches_df.head(5))
        
    
    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection
        conn.close()
    return

# Call the main function (no need for asyncio in this context)
main("Do you have a toy set that teaches numbers and letters to kids?",25,100)

                                        product_name  list_price  \
0                    12"-20" Schwinn Training Wheels       28.17   
1       Slip N Slide Triple Racer with Slide Boogies       37.21   
2  Polaris 39-310 5-Liter Zippered Super Bag for ...       39.47   
3  Sandbox Castle 2-in-1 Sand and Water Table wit...       60.49   
4  Jensen S100T Commercial Tot Full Bucket Rubber...       90.18   

                                         description  
0  Turn any small bicycle into an instrument for ...  
1  Triple Racer Slip and Slide with Boogie Boards...  
2  Keep your pool water sparkling clean all seaso...  
3  Package Includes Sandbox Castle 2-in-1 Sand an...  
4  This is a fully enclosed one piece infant seat...  


In [13]:
# Using LangChain for summarization and efficient context building.

from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from IPython.display import display, Markdown

llm = VertexAI()

map_prompt_template = """
              You will be given a detailed description of a toy product.
              This description is enclosed in triple backticks (```).
              Using this description only, extract the name of the toy,
              the price of the toy and its features.

              ```{text}```
              SUMMARY:
              """
map_prompt = PromptTemplate(template=map_prompt_template, input_variables=["text"])

combine_prompt_template = """
                You will be given a detailed description different toy products
                enclosed in triple backticks (```) and a question enclosed in
                double backticks(``).
                Select one toy that is most relevant to answer the question.
                Using that selected toy description, answer the following
                question in as much detail as possible.
                You should only use the information in the description.
                Your answer should include the name of the toy, the price of the toy
                and its features. Your answer should be less than 200 words.
                Your answer should be in Markdown in a numbered list format.


                Description:
                ```{text}```


                Question:
                ``{user_query}``


                Answer:
                """
combine_prompt = PromptTemplate(
    template=combine_prompt_template, input_variables=["text", "user_query"]
)

docs = [Document(page_content=str(t)) for t in matches]
chain = load_summarize_chain(
    llm, chain_type="map_reduce", map_prompt=map_prompt, combine_prompt=combine_prompt
)
answer = chain.run(
    {
        "input_documents": docs,
        "user_query": user_query,
    }
)


display(Markdown(answer))

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

 1. **Toy Name**: Sandbox Castle 2-in-1 Sand and Water Table with Beach Playset

2. **Price**: $60.49

3. **Features**:

- Includes 14-piece sand tools such as watering can, shovel, rake, castle and shell sand molds, sailboat, and bridges.
- Great activity for indoor or outdoor play and small enough to take on trips.
- Colorful, easy to assemble sand water play table with dual sink design.

In [14]:
# Please fill in these values.
creative_prompt = "A bicycle with brand name 'Roadstar bike' for kids that comes with training wheels and helmet."  # @param {type:"string"}

In [15]:
import psycopg2
from psycopg2 import sql
from pgvector.psycopg2 import register_vector

def main():
    try:
        # AlloyDB cluster connection details (replace with your actual values)
        cluster_ip_address = "10.111.0.2"
        database_user = "postgres"
        database_password = "postgres"
        
        # Connect to AlloyDB cluster
        conn = psycopg2.connect(
            host=cluster_ip_address,
            user=database_user,
            password=database_password)

        # Register the vector type
        register_vector(conn)

        # Get the query embedding
        qe = embeddings_service.embed_query([creative_prompt])
        qe_str = "[%s]" % (",".join([str(x) for x in qe]))
        matches = []
        similarity_threshold = 0.7

        # Perform the similarity search and filtering
        cur = conn.cursor()
        
        # Pass 'qe' twice to match the number of placeholders in the query
        cur.execute(
            """
            WITH vector_matches AS (
                SELECT product_id, 1 - (embedding <=> %s::vector) AS similarity
                FROM product_embeddings
                WHERE 1 - (embedding <=> %s::vector) > %s
                ORDER BY similarity DESC
                LIMIT 1
            )
            SELECT description FROM products
            WHERE product_id IN (SELECT product_id FROM vector_matches)
            """,
            (qe, qe, similarity_threshold))
        results = cur.fetchall()
        # Process the results
        for r in results:
            matches.append(r[0])

        print(matches)
        
    except Exception as e:
        print(f"Error during database operations: {e}")
    finally:
        # Close the connection
        conn.close()
    return

# Call the main function (no need for asyncio in this context)
main()

['Turn any small bicycle into an instrument for learning to ride with the Schwinn 12"-20" Training Wheels. They feature a slotted design to fit 12" to 20" bikes. The training wheels are easy to assemble, install and remove, so that when your little one is able to ride without assistance, you can take them off. These bicycle training wheels include steel brackets and rubber tires that can stand up to heavy use. Training Wheels, Fits 12 inches - 20 inches bicycles. Est. 1895. Durable Construction: Steel brackets stand up to heavy use. Customizable: Two sets of wheel decals included. Features: Fits Most Childrens Bicycles: Intended for 12 inch - 20 inch bicycles. Steel Brackets: Offer increased durability. Includes two sets of wheel decals: Learn how to ride in style - see images below. Easy to Adjust: Slotted design for size adjustment. Includes: One pair of training wheels, four decals, installation instructions, and all mounting hardware. Tools required: Adjustable wrench. www.schwinnb

In [16]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from IPython.display import display, Markdown

template = """
            You are given descriptions about some similar kind of toys in the context.
            This context is enclosed in triple backticks (```).
            Combine these descriptions and adapt them to match the specifications in
            the initial prompt. All the information from the initial prompt must
            be included. You are allowed to be as creative as possible,
            and describe the new toy in as much detail. Your answer should be
            in markdown in lists and less than 200 words.

            Context:
            ```{context}```

            Initial Prompt:
            {creative_prompt}

            Answer:
        """

prompt = PromptTemplate(
    template=template, input_variables=["context", "creative_prompt"]
)

# Increase the `temperature` to allow more creative writing freedom.
llm = VertexAI(temperature=0.7)

# Assuming each dictionary in `matches` has a `description` key:
context = "\n".join(
    match["description"] for match in matches if isinstance(match, dict)
)

llm_chain = LLMChain(prompt=prompt, llm=llm)
answer = llm_chain.run(
    {
        "context": context,
        "creative_prompt": creative_prompt,
   }
)

display(Markdown(answer))

 **Roadstar Bike for Kids with Training Wheels and Helmet**

         * **Durable Construction:** Sturdy steel frame with a scratch-resistant finish.
         * **Adjustable Seat and Handlebars:** Easily adjust the seat and handlebars to fit your child's height.
         * **12-20 Inch Training Wheels:** The included training wheels provide stability for young riders, and they're removable when your child is ready to ride without them.
         * **Quick-Release Seat Post:** Easily adjust the seat height without tools.
         * **Front and Rear Handbrakes:** Provide sure stopping power.
         